<font size=5>**神经网络背向传播的矩阵形式普适描述（含多样本与梯度下降方法）**</font>

本文档以矩阵形式描述具有任意 $L$ 层（包括输入层、若干隐藏层和输出层）的全连接神经网络的前向传播和后向传播过程，涵盖多样本场景下的矩阵维度、运算规则和通用的背向传播算法。特别地，描述包括批量梯度下降（Batch Gradient Descent）、随机梯度下降（Stochastic Gradient Descent, SGD）和小批量梯度下降（Mini-batch Gradient Descent）的实现方式。

# 1. 网络结构与符号定义

- 层数：网络共有 $L$ 层，从 $l = 0$（输入层）到 $l = L-1$（输出层）。层 $l = 1, \dots, L-2$ 为隐藏层。
- 每层神经元数：第 $l$ 层有 $n_l$ 个神经元（输入层为特征数）。
- 样本数：训练数据集包含 $N$ 个样本。对于批量处理，定义批量大小为 $B$（$1 \leq B \leq N$）。
- 输入：输入矩阵 $\boldsymbol{X} = [\boldsymbol{x}_1, \boldsymbol{x}_2, \dots, \boldsymbol{x}_B] = \boldsymbol{A}^{(0)}$，维度 $n_0 \times B$，每列 $\boldsymbol{x}_i$ 为一个样本（维度 $n_0 \times 1$）。
- 输出：预测输出矩阵 $\hat{\boldsymbol{Y}} = \boldsymbol{A}^{(L-1)}$，维度 $n_{L-1} \times B$，每列 $\hat{\boldsymbol{y}}i$ 为一个样本的预测（维度 $n_{L-1} \times 1$）。
- 真实标签：标签矩阵 $\boldsymbol{Y} = [\boldsymbol{y}_1, \boldsymbol{y}_2, \dots, \boldsymbol{y}_B]$，维度 $n_{L-1} \times B$，每列 $\boldsymbol{y}i$ 为一个样本的标签（维度 $n_{L-1} \times 1$）。
- 权重：从第 $l-1$ 层到第 $l$ 层的权重矩阵 $\boldsymbol{W}^{(l)}$，维度 $n_l \times n_{l-1}$。
- 偏置：第 $l$ 层的偏置向量 $\boldsymbol{b}^{(l)}$，维度 $n_l \times 1$。
- 预激活值：第 $l$ 层的输入矩阵 $\boldsymbol{Z}^{(l)}$，维度 $n_l \times B$，每列为一个样本的预激活值。
- 激活值：第 $l$ 层的输出矩阵 $\boldsymbol{A}^{(l)} = \sigma^{(l)}(\boldsymbol{Z}^{(l)})$，维度 $n_l \times B$，每列为一个样本的激活值。
- 激活函数：$\sigma^{(l)}$，逐元素应用（如 sigmoid、ReLU，输出层可能为恒等函数）。
- 损失函数：均方误差（MSE），对 $B$ 个样本求平均：

$$L = \frac{1}{2B} \sum_{i=1}^B ||\boldsymbol{y}_i - \hat{\boldsymbol{y}}_i||_2^2 = \frac{1}{2B} \sum_{i=1}^B \sum_{j=1}^{n_{L-1}} (y_{ji} - \hat{y}_{ji})^2$$

- 逐元素乘法：用 $\odot$ 表示。

# 2. 前向传播（多样本）

前向传播对批量输入 $\boldsymbol{X}$（$n_0 \times B$）逐层计算输出。
对每一层 $l = 1, 2, \dots, L-1$：

- 1. 预激活值：$$\boldsymbol{Z}^{(l)} = \boldsymbol{W}^{(l)} \boldsymbol{A}^{(l-1)} + \boldsymbol{b}^{(l)} \boldsymbol{1}_B^\top$$

    - 维度：
        - $\boldsymbol{W}^{(l)}$：$n_l \times n_{l-1}$
        - $\boldsymbol{A}^{(l-1)}$：$n_{l-1} \times B$
        - $\boldsymbol{W}^{(l)} \boldsymbol{A}^{(l-1)}$：$(n_l \times n_{l-1}) \cdot (n_{l-1} \times B) = n_l \times B$
        - $\boldsymbol{b}^{(l)}$：$n_l \times 1$
        - $\boldsymbol{1}_B$：$B \times 1$（全 1 向量）
        - $\boldsymbol{b}^{(l)} \boldsymbol{1}_B^\top$：$n_l \times B$（偏置广播到每个样本）
        - 结果 $\boldsymbol{Z}^{(l)}$：$n_l \times B$
    
    - 说明：矩阵乘法对批量样本并行计算，偏置通过广播加到每个样本。

- 2. 激活值：$$\boldsymbol{A}^{(l)} = \sigma^{(l)}(\boldsymbol{Z}^{(l)})$$

    - 维度：$\boldsymbol{A}^{(l)}$：$n_l \times B$
    - 说明：激活函数逐元素作用于矩阵，维度不变。

- 3. 输出层：

    - 最后一层 $l = L-1$ 的输出为 $\hat{\boldsymbol{Y}} = \boldsymbol{A}^{(L-1)}$。
    - 计算损失：$$L = \frac{1}{2B} ||\boldsymbol{Y} - \boldsymbol{A}^{(L-1)}||_F^2$$（$||\cdot||_F$ 为 Frobenius 范数）。

# 3. 后向传播（多样本）

后向传播计算损失对每一层权重和偏置的梯度，考虑批量样本的平均梯度。

### 3.1. 输出层误差项

对输出层 $l = L-1$，定义误差项矩阵：$$\boldsymbol{\Delta}^{(L-1)} = \frac{\partial L}{\partial \boldsymbol{A}^{(L-1)}} \odot \sigma^{(L-1)'}(\boldsymbol{Z}^{(L-1)})$$

- 计算：
损失对输出的梯度：$$\frac{\partial L}{\partial \boldsymbol{A}^{(L-1)}} = - \frac{1}{B} (\boldsymbol{Y} - \boldsymbol{A}^{(L-1)})$$
    - 维度：$n_{L-1} \times B$
    - 说明：因子 $\frac{1}{B}$ 确保批量样本的损失平均。


- 激活函数导数：$$\sigma^{(L-1)'}(\boldsymbol{Z}^{(L-1)})$$
    - 维度：$n_{L-1} \times B$


- 误差项：$$\boldsymbol{\Delta}^{(L-1)} = - \frac{1}{B} (\boldsymbol{Y} - \boldsymbol{A}^{(L-1)}) \odot \sigma^{(L-1)'}(\boldsymbol{Z}^{(L-1)})$$
    - 维度：$n_{L-1} \times B$

### 3.2. 隐藏层误差项

对每一隐藏层 $l = L-2, L-3, \dots, 1$，误差项矩阵递归计算：$$\boldsymbol{\Delta}^{(l)} = [(\boldsymbol{W}^{(l+1)})^\top \boldsymbol{\Delta}^{(l+1)}] \odot \sigma^{(l)'}(\boldsymbol{Z}^{(l)})$$

- 计算：
    - 误差传播：$$(\boldsymbol{W}^{(l+1)})^\top \boldsymbol{\Delta}^{(l+1)}$$
    - 维度：
        - $\boldsymbol{W}^{(l+1)}$：$n_{l+1} \times n_l$
        - $(\boldsymbol{W}^{(l+1)})^\top$：$n_l \times n_{l+1}$
        - $\boldsymbol{\Delta}^{(l+1)}$：$n_{l+1} \times B$
        - 结果：$(n_l \times n_{l+1}) \cdot (n_{l+1} \times B) = n_l \times B$

- 激活函数导数：$$\sigma^{(l)'}(\boldsymbol{Z}^{(l)})$$
    - 维度：$n_l \times B$


- 误差项：$$\boldsymbol{\Delta}^{(l)} = [(\boldsymbol{W}^{(l+1)})^\top \boldsymbol{\Delta}^{(l+1)}] \odot \sigma^{(l)'}(\boldsymbol{Z}^{(l)})$$
    - 维度：$n_l \times B$

### 3.3. 权重和偏置的梯度
对每一层 $l = 1, 2, \dots, L-1$：

- 权重梯度：$$\frac{\partial L}{\partial \boldsymbol{W}^{(l)}} = \boldsymbol{\Delta}^{(l)} (\boldsymbol{A}^{(l-1)})^\top$$

    - 维度：
        - $\boldsymbol{\Delta}^{(l)}$：$n_l \times B$
        - $\boldsymbol{A}^{(l-1)}$：$n_{l-1} \times B$
        - $(\boldsymbol{A}^{(l-1)})^\top$：$B \times n_{l-1}$
        - 结果：$(n_l \times B) \cdot (B \times n_{l-1}) = n_l \times n_{l-1}$
    
    - 说明：矩阵乘法对批量样本的梯度求和，生成与 $\boldsymbol{W}^{(l)}$ 相同维度的梯度矩阵。


- 偏置梯度：$$\frac{\partial L}{\partial \boldsymbol{b}^{(l)}} = \boldsymbol{\Delta}^{(l)} \boldsymbol{1}_B$$

    - 维度：
        - $\boldsymbol{\Delta}^{(l)}$：$n_l \times B$
        - $\boldsymbol{1}_B$：$B \times 1$
        - 结果：$(n_l \times B) \cdot (B \times 1) = n_l \times 1$

- 说明：对批量样本的误差求和，生成偏置梯度。

### 3.4. 参数更新

使用梯度下降更新参数：

$$\boldsymbol{W}^{(l)} \leftarrow \boldsymbol{W}^{(l)} - \eta \frac{\partial L}{\partial \boldsymbol{W}^{(l)}}$$

$$\boldsymbol{b}^{(l)} \leftarrow \boldsymbol{b}^{(l)} - \eta \frac{\partial L}{\partial \boldsymbol{b}^{(l)}}$$

其中，$\eta$ 为学习率。

# 4. 梯度下降方法

在多样本场景下，梯度下降方法根据批量大小 $B$ 分为以下三种：

### 4.1 批量梯度下降（Batch Gradient Descent）：

- 批量大小：$B = N$（使用全部训练样本）。
- 特点：
    - 计算所有样本的平均梯度，更新参数：
      $$\frac{\partial L}{\partial \boldsymbol{W}^{(l)}} = \frac{1}{N} \boldsymbol{\Delta}^{(l)} (\boldsymbol{A}^{(l-1)})^\top, \quad \frac{\partial L}{\partial \boldsymbol{b}^{(l)}} = \frac{1}{N} \boldsymbol{\Delta}^{(l)} \boldsymbol{1}_N$$
    - 梯度估计精确，但计算成本高，内存需求大。

- 适用场景：数据集较小，或需要稳定的梯度方向。

### 4.2 随机梯度下降（Stochastic Gradient Descent, SGD）：

- 批量大小：$B = 1$（每次随机选择一个样本）。
- 特点：
    - 对单个样本计算梯度，更新参数：
      $$\frac{\partial L}{\partial \boldsymbol{W}^{(l)}} = \boldsymbol{\delta}^{(l)} (\boldsymbol{a}^{(l-1)})^\top, \quad \frac{\partial L}{\partial \boldsymbol{b}^{(l)}} = \boldsymbol{\delta}^{(l)}$$
      （$\boldsymbol{\delta}^{(l)}$ 为单样本误差项，维度 $n_l \times 1$）。
    - 梯度噪声大，更新频繁，可能逃离局部最小值，但收敛路径波动。

- 适用场景：大数据集，需快速迭代。

### 4.3 小批量梯度下降（Mini-batch Gradient Descent）：

- 批量大小：$1 < B < N$（每次随机选择一小批样本）。
- 特点：
    - 对批量样本计算平均梯度，更新参数，如 3.3 节所述。
    - 平衡了批量梯度下降的稳定性和随机梯度下降的计算效率。
    - 批量大小 $B$ 通常为 32、64、128 等（视硬件和任务调整）。

- 适用场景：现代深度学习中最常用的方法，适合大数据集和 GPU 并行计算。

### 4.4 批量选择与实现：

- 在训练中，数据集通常被划分为多个小批量（mini-batches）。每个 epoch 遍历所有小批量，随机打乱样本顺序以减少偏差。
- 矩阵运算（尤其 $\boldsymbol{W}^{(l)} \boldsymbol{A}^{(l-1)}$ 和 $\boldsymbol{\Delta}^{(l)} (\boldsymbol{A}^{(l-1)})^\top$）天然支持批量处理，适合 GPU 加速。

# 5. 算法流程

- 1. 初始化：随机初始化所有 $\boldsymbol{W}^{(l)}$ 和 $\boldsymbol{b}^{(l)}$。
- 2. 数据准备：将数据集划分为小批量（批量大小 $B$），随机打乱样本顺序。
- 3. 对每个 epoch：
    - 对每个小批量（$\boldsymbol{X}$, $\boldsymbol{Y}$，维度 $n_0 \times B$ 和 $n_{L-1} \times B$）：
    - 前向传播：
        - 对 $l = 1, \dots, L-1$，计算：$$\boldsymbol{Z}^{(l)} = \boldsymbol{W}^{(l)} \boldsymbol{A}^{(l-1)} + \boldsymbol{b}^{(l)} \boldsymbol{1}_B^\top, \quad \boldsymbol{A}^{(l)} = \sigma^{(l)}(\boldsymbol{Z}^{(l)})$$
        - 计算损失 $L$。

    - 后向传播：
        - 计算输出层误差项：
          $$\boldsymbol{\Delta}^{(L-1)} = - \frac{1}{B} (\boldsymbol{Y} - \boldsymbol{A}^{(L-1)}) \odot \sigma^{(L-1)'}(\boldsymbol{Z}^{(L-1)})$$
        - 对 $l = L-2, \dots, 1$，递归计算：
          $$\boldsymbol{\Delta}^{(l)} = [(\boldsymbol{W}^{(l+1)})^\top \boldsymbol{\Delta}^{(l+1)}] \odot \sigma^{(l)'}(\boldsymbol{Z}^{(l)})$$
        - 计算梯度：
          $$\frac{\partial L}{\partial \boldsymbol{W}^{(l)}} = \boldsymbol{\Delta}^{(l)} (\boldsymbol{A}^{(l-1)})^\top, \quad \frac{\partial L}{\partial \boldsymbol{b}^{(l)}} = \boldsymbol{\Delta}^{(l)} \boldsymbol{1}_B$$

    - 更新参数：按选定的梯度下降方法（批量、随机或小批量）更新 $\boldsymbol{W}^{(l)}$ 和 $\boldsymbol{b}^{(l)}$。

- 4. 迭代：重复 epoch，直到损失收敛或达到最大迭代次数。

In [2]:
# 导入类库
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib qt

# 定义类
class MultilayerPerceptron:
    """
    这是一个多层感知机类

    属性：
        data：输入数据，要求是一个大小为 n0xN 的 array 数组，其中 N 表示样本数，n0 表示变量数。
            对于每一个样本，输入都是一个列向量，与通常的机器学习样本不同
        labels：数据标签，要求是一个大小为 n_{L:-1}xN 的 array 数组，其中 n_{L:-1} 表示第 (L-1) 层的
            变量数，N 表示样本数
        size：神经网络各层大小，是一个 list，size = [n0, n1, ..., n_{L-1}]
        num_sample：样本数
        
    方法：
        
    """
    def __init__(self, data, labels, size, normlized=True):
        """
        初始化 MultilayerPerceptron 类的实例

        参数：
            data: 输入数据，要求是一个大小为 mxn0 的 array 数组，其中 m 表示样本数，n0 表示变量数
            labels: 数据标签，要求是一个大小为 mx1 的 array 数组，其中 m 表示样本数
            size: 神经网络各层大小，是一个 list，size = [n0, n1, ..., n_{L-1}]
            normlized: 是否归一化，默认为 True
        """
        W, b = self.theta_init(size) 
        self.W, self.b = W, b # 参数初始化
        
        if normlized:
            data = data/255.
        
        self.normlized = normlized
        self.data = data
        self.labels = labels
        self.size = size
        self.num_sample = data.shape[0] 
        

    def train(self, epoch=1000, eta=0.1, batch_size = 128, test_data=None):
        """
        训练模型

        参数：
            epoch: 循环次数。默认为 1000
            eta: 学习率，用于更新权重系数。默认为 0.1
            batch_size: 小批量梯度下降的样本大小，默认为 128.当此值为 1 时，对应随机梯度下降，
                        当此值与样本数相同时，对应批量梯度下降
            test_data: 测试集，不拆分标签和变量，大小为 (m+1)xn 的 narray。其中 m 是变量数，
                        m+1 是数据标签，n 是样本数。默认为 None，即不测试。     

        返回：
            W：权重矩阵，是一个长度为 L-1 的字典，其中 W[l] 表示第 l-1 到第 l 层传播时的权重矩阵
            b：层间传播的偏置项，是一个长度为 L-1 的字典，其中 b[l] 表示第 l-1 层到第 l 层传播时的偏置系数
            loss：损失函数
        """
        W, b = self.W, self.b
        loss = []
        if test_data is not None: train_score = []
        plt.ion()
        
        for epoch_index in range(epoch):
            shuffled_index = np.random.permutation(self.data.shape[1])
            shuffled_data = [self.data[:, shuffled_index[ii:ii+batch_size]] 
                             for ii in range(0, shuffled_index.size, batch_size)]
            shuffled_labels = [self.labels[shuffled_index[ii:ii+batch_size]] 
                               for ii in range(0, shuffled_index.size, batch_size)]
            
            for mini_data, mini_labels in list(zip(shuffled_data, shuffled_labels)):
            
                # 前向传播
                Z,A,_ = self.forward_propagation(mini_data, W, b)
                
                # 后向传播
                Delta = self.back_propagation(mini_labels, W, Z, A)
                
                # 参数更新
                for layer_index in np.arange(len(W))+1:
                    W[layer_index] -= eta*Delta[layer_index]@A[layer_index-1].T
                    b[layer_index] -= eta*Delta[layer_index]@np.ones((Delta[layer_index].shape[1],1))
                
            if test_data is not None:
                train_score.append(self.test_score(test_data, W, b))
                plt.clf()
                plt.plot(train_score)
                plt.xlabel('Epoches')
                plt.ylabel('Accuracy Rate')
                plt.xlim((0, epoch))
                plt.pause(0.01)
            else:
                loss.append(MultilayerPerceptron.get_mse(self.data, W, b, self.labels, size))
                plt.clf()
                plt.plot(loss)
                plt.xlabel('Epoches')
                plt.ylabel('MSE')
                plt.xlim((0, epoch))
                plt.pause(0.01)
                         
        plt.ioff()
        plt.show()
        self.W, self.b = W, b      
        

    @staticmethod  
    def get_mse(data, W, b, labels, size):
        """
        训练集上的均方差
        
        变量:
            data: 抽选出来的样本值
            W: 初始化的权重矩阵，是一个长度为 L-1 的字典，其中 W[l] 表示第 l-1 到第 l 层传播时的权重矩阵
            b：初始化的层间传播的偏置项，是一个长度为 L-1 的字典，其中 b[l] 表示第 l-1 层到第 l 层传播时的偏置系数
            labels: Bx1 大小的 narray
        返回：
            mse: 均方差
        """
        
        one_hot_labels = MultilayerPerceptron.one_hot(labels)
        B = labels.size
        _, _, Y_hat = MultilayerPerceptron.forward_propagation(data, W, b)
        mse = np.sum((Y_hat - one_hot_labels)**2)/B
        
        return mse
    
    
    def test_score(self, test_data, W, b):
        """
        训练集上的均方差
        
        变量:
            test_data: 测试集，不拆分标签和变量，大小为 (m+1)xn 的 narray。其中 m 是变量数，
                        m+1 是数据标签，n 是样本数。默认为 None，即不测试。     
            W: 训练过程中的权重值
            b：训练过程中的标签值
        返回：
            score: 在测试集上的预测准确率
        """
        
        Y = test_data[0,:]
        B = test_data.shape[1]
        if self.normlized:
            test_data = test_data[1:,:]/255.0
        else:
            test_data = test_data[1:,:]
        _, _, Y_hat = MultilayerPerceptron.forward_propagation(test_data, W, b)
        Y_predict = np.argmax(Y_hat, axis=0)
        score = np.sum(Y==Y_predict)/B
        
        return score
    
    
    @staticmethod  
    def one_hot(labels):
        """
        将 0-9 之间的数字标签转换为 one-hot 格式的标签
        
        变量:
            labels: Bx1 大小的 narray
            
        返回：
            one_hot_labels: 10xB 大小的 narray
        """
        
        B = labels.size
        one_hot_labels = np.zeros((10, B))
        
        for sample_index in range(B):
            one_hot_labels[labels[sample_index], sample_index] = 1
            
        return one_hot_labels


    @staticmethod  
    def back_propagation(labels, W, Z, A):
        """
        前向传播
    
        参数：
            labels: 输入数据的标签，大小为 n_0xB 的 narray 结构。对于数字识别问题，标签是一个数字。
            W: 初始化的权重矩阵，是一个长度为 L-1 的字典，其中 W[l] 表示第 l-1 到第 l 层传播时的权重矩阵
            Z：字典类型，长度为 (L-1)，Z[l] 里面是第 l 层的输入值，是一个大小为 n_lxB 的 narray 格式的数组
            A：字典类型，长度为 (L-1)，A[l] 里面是第 l 层输入值经过激活函数的非线性变换后的输出值，是一个大小
                为 n_lxB 的 narray 格式的数组

        返回：
            Delta: 字典类型，长度为 (L-1)，Delta[l] 里面是第 l 层的误差矩阵，对应损失函数对第 l 层输入值 Z_l 
                的偏导 \frac{\partial L}{\partial Z}，大小为 n_lxB
            
        """
        
        Y = MultilayerPerceptron.one_hot(labels) # 将 labels 转为 one-hot 格式
        Delta, batch_size = {}, labels.size
        for layer_index in np.arange(len(W), 0, -1):
            if layer_index == len(W): # 最后一层
                Delta[layer_index] = -(Y - A[layer_index]) \
                *MultilayerPerceptron.sigmoid_gradient(Z[layer_index])/batch_size
            else:
                Delta[layer_index] = W[layer_index+1].T@Delta[layer_index+1] \
                *MultilayerPerceptron.sigmoid_gradient(Z[layer_index])

        return Delta
     
    
    @staticmethod            
    def forward_propagation(data, W, b):
        """
        前向传播

        参数：
            data：输入数据，要求是一个大小为 mxn0 的 array 数组，其中 m 表示样本数，n0 表示变量数
            W：初始化的权重矩阵，是一个长度为 L-1 的字典，其中 W[l] 表示第 l-1 到第 l 层传播时的权重矩阵
            b：初始化的层间传播的偏置项，是一个长度为 L-1 的字典，其中 b[l] 表示第 l-1 层到第 l 层传播时的偏置系数
            size：神经网络各层大小，是一个 list，size = [n0, n1, ..., n_{L-1}]

        返回：
            Z：字典类型，长度为 L。其中, Z[0] 即是输入数据。Z[l] 里面是第 l 层的输入值，是一个大小为 n_lxB 的 narray 格式的数组
            A：字典类型，长度为 L。其中, A[0] 即是输入数据。A[l] 里面是第 l 层输入值经过激活函数的非线性变换后的输出值，是一个大小
                为 n_lxB 的 narray 格式的数组
        """
        Z, A = {}, {}
        Z[0] = data.copy()
        A[0] = data.copy()
        for layer_index in np.arange(len(W))+1:
            Z[layer_index] = W[layer_index]@A[layer_index-1] + b[layer_index]
            A[layer_index] = MultilayerPerceptron.sigmoid(Z[layer_index])
        
        return Z, A, A[len(W)]
    
    @staticmethod
    def theta_init(size):
        """
        参数初始化

        参数：
            size：模型的各层参数

        返回：
            W：初始化的权重矩阵，是一个长度为 L-1 的字典，其中 W[l] 表示第 l-1 到第 l 层传播时的权重矩阵
            b：初始化的层间传播的偏置项，是一个长度为 L-1 的字典，其中 b[l] 表示第 l-1 层到第 l 层传播时的偏置系数
        """
        W, b = {}, {}
        for layer_index in range(len(size)-1):
            W[layer_index+1] = np.random.randn(size[layer_index+1], size[layer_index])*0.1
            b[layer_index+1] = np.random.randn(size[layer_index+1], 1)*0.1
        return W,b
        
    @staticmethod
    def sigmoid(z):
        return 1.0/(1.0 + np.exp(-z))
        
    @staticmethod
    def sigmoid_gradient(z):
        return MultilayerPerceptron.sigmoid(z)*(1 - MultilayerPerceptron.sigmoid(z))

    
#################################################################################################
# 读入图片数据
data = pd.read_csv(r"D:\DeskTop\Code\MyPython\西瓜书\3-线性模型\data\digits.csv")
        
# 数据预处理
train_data = data.sample(frac=0.8)
test_data = data.sample(10000).values.T

num_training_examples = 6000

x_train = train_data.iloc[:num_training_examples, 1:].values.T
y_train = train_data.iloc[:num_training_examples, 0].values

# 训练模型
size = [784, 40, 10]
max_iterations = 100
eta = 1

multilayer_perceptron = MultilayerPerceptron(x_train, y_train, size, normlized=True)
multilayer_perceptron.train(epoch=max_iterations, eta=eta, 
                            batch_size=200, test_data=test_data)